In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [65]:
from fastai.vision import *
from fastai.metrics import accuracy


from torch.autograd import Variable
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform

In [3]:
PATH = "/scratch/as3ek/chrc_data_case_preserved_1000_normal/"
sz=512
tfms = get_transforms(do_flip=True, flip_vert=True, max_zoom=1.1)

In [4]:
data = (ImageItemList.from_folder(PATH)
        .split_by_folder()          
        .label_from_folder()
        .transform(tfms, size=sz)
        .databunch(bs=16)
        .normalize(imagenet_stats))

In [5]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    
    def forward(self, x): 
        return torch.cat([self.mp(x), self.ap(x)], 1)
    
    
custom_head = nn.Sequential(
                            AdaptiveConcatPool2d(),
                            Flatten(), 
                            nn.BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                            nn.Dropout(p=0.25),
                            nn.Linear(in_features=4096, out_features=512, bias=True),
                            nn.ReLU(),
                            nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                            nn.Dropout(p=0.5),
                            nn.Linear(in_features=512, out_features=10, bias=True),
                            nn.ReLU(),
                            nn.Linear(in_features=10, out_features=3, bias=True),
                            nn.LogSoftmax()
                           )

In [6]:
learn = create_cnn(data, models.resnet50, metrics=accuracy, custom_head=custom_head)

In [9]:
learn.unfreeze()

In [10]:
learn.fit_one_cycle(1, max_lr=slice(1e-6,1e-3))

epoch,train_loss,valid_loss,accuracy
1,0.216016,0.467687,0.817645


/home/as3ek/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [11]:
learn.save('resnet_10_output_1_epoch')

In [102]:
learn.load('resnet_10_output_1_epoch');
learn.unfreeze()

In [103]:
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-3))

epoch,train_loss,valid_loss,accuracy
1,0.172274,0.519888,0.794384
2,0.087812,0.476235,0.821145


/home/as3ek/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [32]:
class ResNet50Bottom(nn.Module):
    def __init__(self, original_model):
        super(ResNet50Bottom, self).__init__()
        self.features = nn.Sequential(*(list(learn.model.children())[:-1] + list(list(learn.model.children())[1])[:-3]))
        
    def forward(self, x):
        x = self.features(x)
        return x

model_10_features = ResNet50Bottom(learn.model)

In [82]:
train_df = data.train_ds.to_df()

In [87]:
bs = 16
data_model_trn = pd.DataFrame(columns = list(range(10)).append('fname'))

dl_iter = iter(learn.data.train_dl)
dl = next(dl_iter)
for index, name in enumerate(train_df['x']):
    if ((index % bs == 0) and (index != 0)):
        dl = next(dl_iter)
    image_batch, label_batch = dl
    i = index % bs
    image = image_batch[i].reshape(1, 3, 512, 512)
    label = label_batch[i].reshape(-1)
    
    output = model_10_features(image)
    
    if (index % 100 == 0):
        print (index)
    
    a = output[0].cpu().detach().numpy()
    tmp = pd.DataFrame(data=a).T
    tmp['fname'] = [name]
    data_model_trn = pd.concat([data_model_trn, tmp], ignore_index=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

StopIteration: 

In [99]:
data_model_trn.to_csv('resnet_train_10_features.csv', index=False)

In [94]:
valid_df = data.valid_ds.to_df()

In [95]:
bs = 16
data_model_val = pd.DataFrame(columns = list(range(10)).append('fname'))

dl_iter = iter(learn.data.valid_dl)
dl = next(dl_iter)
for index, name in enumerate(valid_df['x']):
    if ((index % bs == 0) and (index != 0)):
        dl = next(dl_iter)
    image_batch, label_batch = dl
    i = index % bs
    image = image_batch[i].reshape(1, 3, 512, 512)
    label = label_batch[i].reshape(-1)
    
    output = model_10_features(image)
    
    if (index % 100 == 0):
        print (index)
    
    a = output[0].cpu().detach().numpy()
    tmp = pd.DataFrame(data=a).T
    tmp['fname'] = [name]
    data_model_val = pd.concat([data_model_val, tmp], ignore_index=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [98]:
data_model_val.to_csv('resnet_valid_10_features.csv', index=False)

In [105]:
data_model_trn.head()

,0,1,2,3,4,5,6,7,8,9,fname
0,-3.020246,-4.827797,2.072683,17.903902,-6.798148,-4.858539,-5.082790,-3.854629,0.257859,-3.184415,train/Celiac/C13-65_04___2000_4432___0_0.jpg
1,-1.703454,-2.075338,10.458097,-6.349043,5.109484,6.991062,-0.342912,2.828005,-6.261195,-3.106742,train/Celiac/C10-52_02___6795_3000___0_250.jpg
2,-2.920479,-2.994539,-5.608110,23.793152,-10.577285,-8.400323,-5.386734,-5.328491,5.370527,-1.343166,train/Celiac/C13-68_05___1000_4000___250_0.jpg
3,-3.927897,-4.186972,-4.920321,28.587124,-8.506809,-5.272788,-6.207527,-5.727844,2.162848,-4.702887,train/Celiac/C10-48_01___14207_2000___250_500.jpg
4,-1.170831,-3.460150,7.270679,-5.516728,3.196418,6.509511,0.837857,-1.874611,-3.726884,-2.920571,train/Celiac/C12-59_00___19000_18000___0_0.jpg
